In [ ]:
# Gerekli paketleri yüklüyoruz
!pip install ultralytics geopandas shapely onnxruntime -q
# Veriyi indir ve klasöre çıkar
# Bina anotasyonları ve görüntüleri içeren Las Vegas veri setini indiriyoruz
!wget https://spacenet-dataset.s3.amazonaws.com/spacenet/SN2_buildings/tarballs/SN2_buildings_train_AOI_2_Vegas.tar.gz -O vegas_buildings.tar.gz

# Çıkarma işlemi için uygun klasörü oluşturup arşivi açıyoruz
!mkdir -p spacenet
!tar -xvzf vegas_buildings.tar.gz -C spacenet

# Çözülmüş veri klasörünü kontrol et
import os

for root, dirs, files in os.walk("./spacenet"):
    print(root, len(files), "dosya")
# GeoJSON -> YOLO Etiket Dönüşüm Fonksiyonu
import geopandas as gpd
from shapely.geometry import box
import os

def geojson_to_yolo(geojson_path, image_size, yolo_dir):
    gdf = gpd.read_file(geojson_path)
    os.makedirs(yolo_dir, exist_ok=True)

    for i, row in gdf.iterrows():
        geom = row.geometry
        if geom.geom_type == "Polygon":
            minx, miny, maxx, maxy = geom.bounds
            x_center = ((minx + maxx) / 2) / image_size[0]
            y_center = ((miny + maxy) / 2) / image_size[1]
            width = (maxx - minx) / image_size[0]
            height = (maxy - miny) / image_size[1]
            with open(f"{yolo_dir}/image_{i}.txt", 'a') as f:
                f.write(f"0 {x_center} {y_center} {width} {height}\n")
### 🗑️ Gereksiz Görüntüleri Silme – Alan Optimizasyonu
import os

image_dir = './spacenet/PS-RGB'
label_dir = './spacenet/labels'

deleted = 0
for img_file in os.listdir(image_dir):
    if img_file.endswith('.jpg'):
        txt_file = img_file.replace('.jpg', '.txt')
        label_path = os.path.join(label_dir, txt_file)

        if not os.path.exists(label_path) or os.path.getsize(label_path) == 0:
            os.remove(os.path.join(image_dir, img_file))
            deleted += 1

print(f"✅ {deleted} gereksiz görüntü silindi.")


# YOLOv8 Eğitim Yapılandırması Dosyası building.yaml
yaml_text = """
train: ./spacenet/images/train
val: ./spacenet/images/val
names:
  0: building
"""

with open("building.yaml", "w") as f:
    f.write(yaml_text)
# YOLOv8 Eğitimi
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="building.yaml", epochs=60, imgsz=640)
model.export(format="onnx")
# ONNX Model Testi
import onnxruntime
import numpy as np

session = onnxruntime.InferenceSession("yolov8n.onnx")
input_name = session.get_inputs()[0].name
dummy_input = np.random.rand(1, 3, 640, 640).astype(np.float32)
output = session.run(None, {input_name: dummy_input})

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46